In [1]:
!pip install transformers

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
# from pytorch_pretrained_bert import BertTokenizer, BertConfig
# from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification

from tqdm import tqdm , trange
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
from transformers import BertModel, BertTokenizer

In [10]:
import pandas as pd
!git clone https://gitlab.com/vibss2397/nlp.git
df = pd.read_csv("nlp/fce_old.csv")

Cloning into 'nlp'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 35 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.
Checking out files: 100% (7/7), done.


# Data Processing Old

In [8]:
!ls nlp

In [ ]:
df1 = df[['incorrect', 'err_type']]
df1 = df1.rename(columns={'incorrect':'input','err_type':'err'})

df2 = df[['correct']]
df2.insert(1, 'err', 'CO')
df2 = df2.rename(columns={'correct':'input','err':'err'})

In [ ]:
frames = [df1, df2]
result = pd.concat(frames)

In [ ]:
result.shape

(89044, 2)

In [ ]:
df3 = result.groupby(by='input')['err'].unique().apply(lambda x: ','.join(x)).reset_index()

In [ ]:
df3['err'] = df3.err.str.split(',')

In [ ]:
def remove_arr(arr, ele):
  if ele in arr:
    arr.remove(ele)
  return arr

df3['err'] = df3['err'].apply((lambda x: ['CO'] if 'CO' in x else x))
df3['err'] = df3['err'].apply(lambda x: remove_arr(x, 'DS')).apply(lambda x: remove_arr(x, 'SM'))

In [ ]:
df3.shape

(17873, 2)

In [ ]:
df3_final = df3[df3['err'].apply(lambda x: len(x)!=0)]

In [ ]:
err_categories = sorted(['LX', 'OT', 'GM', 'CO'])

In [9]:
for err_type in err_categories:
  df3_final[err_type] = 0

In [ ]:
def make_1(row):
  for ele in row[1]:
    row[2 + err_categories.index(ele)] = 1
  return row
df3_final = df3_final.apply(make_1, axis = 1)

In [ ]:
df3_final.shape

(17833, 6)

In [ ]:
df4 = df3_final.drop('err', axis = 1)
# df4.to_csv('processed_fce.csv')

# New Data Processing



In [13]:
df4 = pd.read_csv('nlp/fce_final.csv')
df4.head()

,Unnamed: 0,0,1
0,0,He completed B.Sc.,[1 0 0 0]
1,1,He completed his M.Phil in Gujarati literature...,[1 0 0 0]
2,2,He was appointed as Governor of Gujarat in 1954.,[1 0 0 0]
3,3,Shiva temple 3.,[1 0 0 0]
4,4,4 in 1984.,[1 0 0 0]


In [14]:
#selecting sentence and labels
df4 = df4[['0', '1']]

In [16]:
# Finding the length of the biggest sentence
df4['len'] = df4.apply(lambda x: len(x[0].split(' ')), axis=1)
df5 = df4[['0', 'len']]
df5_numpy = df5.to_numpy()
max_len = ['', float('-inf')]
for ele in df5_numpy:
  if(ele[1]>max_len[1]):
    max_len[0] = ele[0]
    max_len[1] = ele[1]
df4 = df4.drop('len', axis=1)

In [17]:
max_len

['Today I have to tell you something really exciting and also very hard to believe. Last month I received a letter which told me that I\'d enteredI\'d won a competition. I was so happy and surprised that first I could not really believe it.  But it was true. My prize was the chance to meet a pop group and to help at their concert. The pop group which was going to play is called "Metallica" and I like their music very much (\'cause in my opinion the music is a bit different from all the songs you usually listen to). But now to the concert itself. It took place on the 10th of June in Salzburg. It started at 8 pm but I had to be there at 11 am. First I had to help with the catering, which lasted until 2 pm. After this I met the band members and got to know them while we were having a good meal. The front man felt a bit sick and so he skipped the meal. I was so excited that I was not able to eat anything either. At 5 pm the band members disappeared again and I was responsible for some kind

In [ ]:
# Splitting into train/test dataset
train_data = df4.sample(frac=0.80, random_state=0)
test_data = df4.drop(train_data.index)
train_arr = np.array(train_data)
test_arr = np.array(test_data)
print("Train shape", train_arr.shape)
print("Test shape", test_arr.shape)

Train shape (24578, 2)
Test shape (6145, 2)


In [ ]:
# csv contains label array but array is a string(eg : '[0, 1, 0, 0]'), converting string back to arr.
final_train_arr = []
for ele in train_arr:
  final_train_arr.append([ele[0], np.array([int(a) for a in ele[1][1:-1].split(' ')])])
final_test_arr = []
for ele in test_arr:
  final_test_arr.append([ele[0], np.array([int(a) for a in ele[1][1:-1].split(' ')])])

In [ ]:
train_data = pd.DataFrame(final_train_arr)
test_data = pd.DataFrame(final_test_arr)

In [ ]:
# weights for weighted loss.
weights = np.sum(train_data[1])
weights_sum = np.sum(weights)
weights2 = weights_sum/weights
weights2 = weights2/weights2[0]
# weights2 = np.exp(weights2)/sum(np.exp(weights2))
print(weights2)

[1.         3.47405299 2.08598475 4.26964478]


In [ ]:
class CustomBERTModel(nn.Module):
    def __init__(self, n_heads = 4, last_layer_unfreeze = False):
          super(CustomBERTModel, self).__init__()
          self.bert = BertModel.from_pretrained("bert-base-uncased")
          for name, param in self.bert.named_parameters():
            if(last_layer_unfreeze):
              if(name.find('pooler')>=0 or name.find('11')>=0 or name.find('10')>=0):
                param.requires_grad = True
              else:
                param.requires_grad = False
            else:
              param.requires_grad = True

          self.dropout = nn.Dropout(0.5)
          self.out1 = nn.Linear(768, 1)
          self.out2 = nn.Linear(768, 1)
          self.out3 = nn.Linear(768, 1)
          self.out4 = nn.Linear(768, 1)
    def forward(self, input_ids, token_type_ids, attention_mask):
          pooled_output = self.bert(
               input_ids, 
               attention_mask,
               token_type_ids
          )
          out1 = self.out1(pooled_output[1])
          
          out2 = self.out2(pooled_output[1])
          
          out3 = self.out3(pooled_output[1])

          out4 = self.out4(pooled_output[1])
          return out1, out2, out3, out4

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bert_model = CustomBERTModel().to(device)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# uses pretrained weights in the nlp repository downloaded above.
bert_model.load_state_dict(torch.load('nlp/final/bert-model-mh3.pth'))

<All keys matched successfully>

In [ ]:
# Creating Custom Dataset
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, max_len):
        self.data = data
        self.targets = targets
        self.max_len = max_len
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        x = self.data[index]
        label = torch.tensor(self.targets[index], dtype=torch.float)
        return [x, label]
    def __len__(self):
        return len(self.data)

In [ ]:
# Tokenizer encoding
def encode_sentence(sentence, max_len):
    return bert_tokenizer.batch_encode_plus(
                        sentence,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        padding = 'max_length',
                        truncation = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt'    # Return pytorch tensors.
                   )
# converting arr to dataset
train_dataset = MyDataset(train_data[0], train_data[1], max_len[1])

# 50% of test arr is validation dataset, rest is test
validation_dataset = MyDataset(test_data[0].to_numpy()[:len(test_data[0])//2], 
                               test_data[1].to_numpy()[:len(test_data[0])//2], max_len[1])

test_dataset = MyDataset(test_data[0].to_numpy()[len(test_data[0])//2:], 
                               test_data[1].to_numpy()[len(test_data[0])//2:], max_len[1])

In [ ]:
bs = 16
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            shuffle = True,
            batch_size = bs # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            validation_dataset,  # The training samples.
            sampler = SequentialSampler(validation_dataset), # Select batches sequentially
            batch_size = bs # Trains with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset,  # The training samples.
            sampler = SequentialSampler(test_dataset), # Sequentially
            batch_size = bs # Trains with this batch size.
        )

In [ ]:
optimizer = AdamW([
                   {'params': bert_model.bert.parameters(), 'lr': 2e-5},
                   {'params': bert_model.out1.parameters(), 'lr': 1e-4},
                   {'params': bert_model.out2.parameters(), 'lr': 1e-4},
                   {'params': bert_model.out3.parameters(), 'lr': 1e-4},
                   {'params': bert_model.out4.parameters(), 'lr': 1e-4},
                   ],
                  weight_decay=1,
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8.
                   )
# Separate loss function with different weights for each error class.
loss = [nn.BCEWithLogitsLoss(reduction = 'sum', pos_weight=torch.tensor(weights2[0])), nn.BCEWithLogitsLoss(reduction = 'sum', pos_weight=torch.tensor(weights2[1])), 
         nn.BCEWithLogitsLoss(reduction = 'sum', pos_weight=torch.tensor(weights2[2])), nn.BCEWithLogitsLoss(reduction = 'sum', pos_weight=torch.tensor(weights2[3]))]

In [ ]:
# Run this if you want to train a new model.
epochs = 5
MAX_LEN = max_len[1]
for epoch_i in range(0, epochs):
    # Running BERT model on train dataset.
    train_loss = [0, 0]
    for step, batch in enumerate(train_dataloader):
        if step % 100 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,} Loss {:>5,}.'.format(step, len(train_dataloader), train_loss[0]/train_loss[1]))
        inputs = batch[0]
        labels = batch[1]
        inputs_encoded = encode_sentence(inputs, MAX_LEN)
        for ele in inputs_encoded:
            inputs_encoded[ele] = inputs_encoded[ele].to(device)
        inputs_encoded['token_type_ids'] = None

        labels = labels.to(device)
        optimizer.zero_grad()
        logits_arr = bert_model(**inputs_encoded)
        
        curr_batch_size = labels.size()[0]
        curr_loss = 0
        for i in range(len(logits_arr)):
          curr_loss+= loss[i](logits_arr[i], labels[:, i].reshape(curr_batch_size, 1))
        curr_loss.backward()
        train_loss[0]+= curr_loss.data.cpu().item()
        train_loss[1]+= curr_batch_size
        optimizer.step()
    print('Training Loss is : %f'%(train_loss[0]/train_loss[1]))
    # Running BERT model on validation dataset.
    bert_model.eval()
    val_loss = [0, 0]
    for step, batch in enumerate(validation_dataloader):
        inputs = batch[0]
        labels = batch[1]
        inputs_encoded = encode_sentence(inputs, MAX_LEN)
        for ele in inputs_encoded:
            inputs_encoded[ele] = inputs_encoded[ele].to(device)
        inputs_encoded['token_type_ids'] = None
        labels = labels.to(device)
        with torch.no_grad():
            logits_arr = bert_model(**inputs_encoded)
            curr_batch_size = labels.size()[0]
            curr_loss = 0
            for i in range(len(logits_arr)):
              curr_loss+= loss[i](logits_arr[i], labels[:, i].reshape(curr_batch_size, 1))
            val_loss[0]+= curr_loss.data.cpu().item()
            val_loss[1]+= curr_batch_size
    bert_model.train()
    print('Validation Loss is : %f'%(val_loss[0]/val_loss[1]))

In [ ]:
# model = torch.load(MODEL_LOAD_DIR)

In [ ]:
# Getting statistics on Test set
from sklearn.metrics import f1_score, accuracy_score, classification_report
err_label = []
err_logits = []
bert_model.eval()
for i, batch in enumerate(test_dataloader):
  input = batch[0]
  labels = batch[1]
  inputs_encoded = encode_sentence(input, 512)
  curr_loss = 0
  total_batch_size = 0
  for ele in inputs_encoded:
      inputs_encoded[ele] = inputs_encoded[ele].to(device)
  labels = labels.to(device)
  with torch.no_grad():
    logits = bert_model(**inputs_encoded)
    logits_combined = torch.cat(logits, dim=1)
    logits_to_labels = torch.sigmoid(logits_combined)
  err_label.extend(labels.data.cpu().numpy())
  err_logits.extend(logits_to_labels.data.cpu().numpy())
  total_batch_size += labels.size()[0]
  err_predictions = np.array(np.array(err_logits)>0.5, dtype = np.int)
  err_labels = np.array(err_label, dtype=np.int)
print(f1_score(err_labels, err_predictions, average='micro'))
print(accuracy_score(err_labels, err_predictions))
print(classification_report(err_labels, err_predictions, target_names=err_categories))

0.7181201255950572
0.5421412300683371
              precision    recall  f1-score   support

          CO       0.70      0.90      0.79      1549
          GM       0.59      0.72      0.65       935
          LX       0.70      0.70      0.70      1301
          OT       0.62      0.76      0.69       737

   micro avg       0.66      0.78      0.72      4522
   macro avg       0.65      0.77      0.71      4522
weighted avg       0.66      0.78      0.72      4522
 samples avg       0.68      0.78      0.71      4522



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# random stuff

In [ ]:
!nvidia-smi

Tue Nov 10 04:07:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    24W /  75W |   7551MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------